In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from PIL import Image

In [ ]:
# Veri seti
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# Boş listeler oluşturuyoruz
labels = []
paths = []

for dir_name, _, filenames in os.walk(fish_dir):
    # "GT" klasörlerini atlıyoruz
    if 'GT' not in dir_name:
        for filename in filenames:
            if os.path.splitext(filename)[1] == ".png":
                labels.append(os.path.basename(dir_name))
                paths.append(os.path.join(dir_name, filename))

# Verileri DataFrame'e ekleme
data = pd.DataFrame({
    'path': paths,
    'label': labels
})

print(data.head())

görüntüleri eğitim için kullanmak yerine, onları veri setinden ayırarak genellikle eğitim veya test verileriyle karışmaması için yapıldı¶

In [ ]:
# Görüntü boyutlandırma
image_size = (128, 128)

# Görüntüleri ve etiketleri yükleme
X = []
y = []

for path, label in zip(data['path'], data['label']):
    img = Image.open(path).resize(image_size)
    img = np.array(img)
    X.append(img)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Verileri normalize etme
X = X / 255.0

# Etiketleri sayısallaştırma
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [ ]:
# Eğitim ve test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Görüntüleri kullanabilmek için eğitim ve test olarak ayırdık.Öncesinde boyutlandırma işleminden sonra normalize ettik.

In [ ]:

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

datagen.fit(X_train)

Veri artırımı ile daha uygun sonuçlar elde edebileceğimi gözlemledim ve artırım yaptım.

In [ ]:
# Modeli oluşturma
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # Çıkış katmanı

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=20,
                    callbacks=[early_stopping])

# Model performansını değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test kaybı: {loss}")
print(f"Test doğruluğu: {accuracy}")

In [ ]:
# Sonuçları görselleştirme
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()

# Tahminleri yapma
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Sınıflandırma raporunu yazdırma
print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))

# Confusion matrix'i görselleştirme
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()